### Prepping Data Challenge: Restaurant Orders (Week 16)
The Preppin' Data team are planning to go out to dinner. However, the system the restaurant has used to collect orders doesn't provide the chefs with the easiest way to understand what they should be making. To help things go swimmingly on the night, as well as for all other guests using this ordering system in the future, we'll create a workflow to only show the ordered dishes for each guest.

### Requirements
- Input the data
- Reshape the Orders table so that we have 3 columns:
  - Guest name
  - Dish
  - Selections (containing 🗸 or null)
- Extract the course name from the Dish field
  - Group these so that Starter and Starters are treated the same, for example
- Fill down the course name for each Guest
  - It may help to bring in the Recipe ID from the Lookup Table 
  - Where the Dish contains the Course name, it may be helpful to replace the Recipe ID in the following way:
    - Starters = 1
    - Mains = 2 
    - Dessert = 3
- Filter out where the Dish = Course
- Filter out Dishes which have not been selected
- Output the Data

In [1]:
import pandas as pd

In [2]:
# Input the data.
with pd.ExcelFile('WK16-Menu Input.xlsx') as xlsx:
    orders = pd.read_excel(xlsx, 'Orders')
    lookup = pd.read_excel(xlsx, 'Lookup Table')

In [3]:
orders.head(10)

,Carl,Unnamed: 1,Unnamed: 2,Tom,Unnamed: 4,Unnamed: 5,Jenny,Unnamed: 7,Unnamed: 8,Jonathan,Unnamed: 10
0,Starters,NaN,NaN,Starter,NaN,NaN,Starters,NaN,NaN,Starters,NaN
1,Prawn Cocktail,🗸,NaN,Prawn Cocktail,NaN,NaN,Camembert,🗸,NaN,Garlic Mushrooms,NaN
2,Pate,NaN,NaN,Smoked Salmon,🗸,NaN,Hummus & Pita,NaN,NaN,Hummus & Pita,NaN
3,Camembert,NaN,NaN,Camembert,NaN,NaN,Garlic Bread,NaN,NaN,Cauliflower Hot Wings,🗸
4,Mains,NaN,NaN,Mains,NaN,NaN,Main,NaN,NaN,Mains,NaN
5,Duck a l'orange,🗸,NaN,Swordfish,NaN,NaN,Halloumi Burger,NaN,NaN,Jackfruit Bolognese,NaN
6,Steak,NaN,NaN,Cod & Chips,NaN,NaN,Daal,🗸,NaN,Nut Roast,NaN
7,Chicken Chasseur,NaN,NaN,Moules-Frites,🗸,NaN,Salad,NaN,NaN,Salad,NaN
8,Salad,NaN,NaN,Salad,NaN,NaN,Spinach & Ricotta Gnocchi,NaN,NaN,Butternut Squash Risotto,🗸
9,Dessert,NaN,NaN,Dessert,NaN,NaN,Dessert,NaN,NaN,Desserts,NaN


In [11]:
lookup.head()

,Recipe ID,Dish
0,18,Prawn Cocktail
1,24,Pate
2,36,Camembert
3,44,Smoked Salmon
4,48,Hummus & Pita


In [4]:
orders.columns

Index(['Carl', 'Unnamed: 1', 'Unnamed: 2', 'Tom', 'Unnamed: 4', 'Unnamed: 5',
       'Jenny', 'Unnamed: 7', 'Unnamed: 8', 'Jonathan', 'Unnamed: 10'],
      dtype='object')

In [ ]:
#Reshape the Orders table so that we have 3 columns: (Guest name, Dish,Selections (containing 🗸 or null))
#orders = orders[['Carl','Tom','Jenny','Jonathan']]

In [5]:
ord_cols = orders.columns
guest = [g for g in ord_cols if g[0:7]!='Unnamed']
select = {i:j for i,j in zip(ord_cols[:-1], ord_cols[1:]) if i[0:7]!='Unnamed'}

In [6]:
orders['Course'] = orders['Carl'].apply(lambda x: x if x in ['Starters','Mains','Dessert'] else None).ffill()

In [7]:
for g in guest:
    orders[g] = orders.apply(lambda x: x[g] if pd.notna(x[select[g]]) else None, axis=1)

In [8]:
output = orders[['Course']+guest].melt(id_vars='Course',value_name='Dish', var_name='Guest').dropna()
output = output.merge(lookup)[['Course','Guest','Recipe ID','Dish']]
output = output.sort_values(['Guest','Course'], ascending=[True, False])

In [9]:
output.head()

,Course,Guest,Recipe ID,Dish
0,Starters,Carl,18,Prawn Cocktail
1,Mains,Carl,128,Duck a l'orange
2,Dessert,Carl,356,Chocolate Bombe
6,Starters,Jenny,36,Camembert
7,Mains,Jenny,234,Daal


In [10]:
#output the data
output.to_csv('wk16-output.csv', index=False)